In [1]:
import math
from tensorboardX import SummaryWriter
import roboschool
import quadruppedEnv
import env_configurations
sess = None

robot lib not fouund

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers

env_name = 'RoboschoolAnt-v1'#'Pendulum-v0'#'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
print(action_space.low)
print(action_space.high)

[-1. -1. -1. -1. -1. -1. -1. -1.]
[1. 1. 1. 1. 1. 1. 1. 1.]


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-06-07 19:03:00,926	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-07_19-03-00_6665/logs.
2019-06-07 19:03:01,040	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:60073 to respond...
2019-06-07 19:03:01,159	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:47232 to respond...
2019-06-07 19:03:01,167	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-06-07 19:03:01,225	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-07_19-03-00_6665/logs.
2019-06-07 19:03:01,227	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.104',
 'redis_address': '192.168.3.104:60073',
 'object_store_address': '/tmp/ray/session_2019-06-07_19-03-00_6665/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-06-07_19-03-00_6665/sockets/raylet',
 'webui_url': None}

In [ ]:
from a2c_continuous import A2CAgent
import tr_helpers
import networks
import a2c_games_configurations

#a2c_config = a2c_games_configurations.bipedalwalker_config
a2c_config = a2c_games_configurations.roboschoolant_config
#a2c_config = a2c_games_configurations.pendulum_config
#a2c_config = a2c_games_configurations.quadrupped_config
#a2c_config = a2c_games_configurations.mountain_car_cont_config
agent = A2CAgent(sess,'car', obs_space, False, action_space, a2c_config)
#agent.restore('nn/latest')
agent.train()
agent.save('nn/latest')

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


2019-06-07 19:03:02,606	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=6751) 
(pid=6751) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=6751) For more information, please see:
(pid=6751)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=6751)   * https://github.com/tensorflow/addons
(pid=6751) If you depend on functionality not listed there, please file an issue.
(pid=6751) 
(pid=6753) 
(pid=6753) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=6753) For more information, please see:
(pid=6753)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=6753)   * https://github.com/tensorflow/addons
(pid=6753) If you depend on functionality not listed there, please file an issue.
(pid=6753) 
(pid=6759) 
(pid=6759) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=6759) For more information, please see:
(pid=6759)   * https://github.com/tensorflow/community/blob/master/rf

(pid=6758) robot lib not fouund
(pid=6758) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=6758)   result = entry_point.load(False)
(pid=6811) 
(pid=6811) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=6811) For more information, please see:
(pid=6811)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=6811)   * https://github.com/tensorflow/addons
(pid=6811) If you depend on functionality not listed there, please file an issue.
(pid=6811) 
(pid=6802) robot lib not fouund
(pid=6802) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=6802)   result = entry_point.load(False)
(pid=6777) 
(pid=6777) WARNING: The Tens

Frames per seconds:  1830.092140621175
saving next best rewards:  461.9106481406982
Frames per seconds:  1765.5011445015114
saving next best rewards:  470.5111847166109
Frames per seconds:  1773.481824924629
saving next best rewards:  480.19687692024104
Frames per seconds:  1817.2538656341032
saving next best rewards:  487.6924932070647
Frames per seconds:  1946.677974859335
saving next best rewards:  497.54675880751034
Frames per seconds:  1917.1175331441793
saving next best rewards:  512.1281850596212
Frames per seconds:  2020.3565032043123
saving next best rewards:  524.6000365783075
Frames per seconds:  2014.8723084302758
saving next best rewards:  531.5058999435893
Frames per seconds:  1962.9575418648444
saving next best rewards:  542.2295456832757
Frames per seconds:  1969.7571394780975
saving next best rewards:  542.3515769568087
Frames per seconds:  2027.0392558373098
Frames per seconds:  2004.042549016886
Frames per seconds:  1953.5498686631386
saving next best rewards:  543.2

In [ ]:
ray.shutdown()

In [ ]:
import numpy as np
agent.restore('nn/latest')
#agent.restore('nn/rob_20QuadruppedWalk-v1')
#agent.restore('nn/ppo_continuousQuadruppedWalk-v1')
env = env_configurations.a2c_configurations[env_name]['ENV_CREATOR']()
def evaluate(env,t_max=5000):
    rewards = []
    #env._max_episode_steps = 5000
    env.batch_mode = True
    print('reset')
    #env = env.old_env
    s = env.reset()
    reward = 0
    for it in range(5000):
        action = agent.get_action([s], False)
        
        #ad = agent.get_action_distribution([s])
        #print(ad)
        s, r, done, _ = env.step(np.squeeze(action, axis = 0))
        print(action)
        env.render()
        reward += r
        
            
        if done:
            break       
        
    return reward

import gym.wrappers

#env_monitor = wrappers.make_atari_deepmind(env_name, noop_max=30, skip=4)
#env_monitor = wrappers.ReallyDoneWrapper(env_monitor)
#env = gym.wrappers.Monitor(env,directory='video_ppo',force=True)

sessions = [print('reward:', evaluate(env)) for _ in range(1)]
#env.close()




In [ ]:
obs = env.reset()
obs, r, done, _ = env.step(2)

done = False
for _ in range(130):
    obs, _, done, _ = env.step(1)
    #env.render()
    obs = np.array(obs)
    if done:
        print('done')
        break
plt.title("what your network gonna see")
print (obs.shape)
plt.imshow(obs[:,:,0],interpolation='none',cmap='gray');

In [ ]:
import gym
gym.envs.registry.all()